In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import sys
import os
from importlib import reload
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.metrics import r2_score
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from collections import namedtuple
import keras_tuner as kt
import time
import pickle
import scipy

In [1]:
1

1

In [3]:
codebase_path = '/data/home/wpw035/Codebase'
sys.path.insert(0, codebase_path) #add path to my codebase models

In [4]:
#my moudles
from DRP_utils import data_preprocessing as dp_nb
reload(dp_nb)
from DRP_utils import model_selection as ms_nb
reload(ms_nb)
from DRP_utils import testing as t_nb
reload(t_nb)
import Data_imports as di_nb
reload(di_nb)
import pairs_train_test_split as  tts_nb
reload(tts_nb)
import Learning_curve as lc_nb
reload(lc_nb)


<module 'Learning_curve' from '/data/home/wpw035/Drug_response_prediction/DRP-alpha-preliminary-results/Unseen_cell_line_testing/Learning_curve.py'>

In [5]:
#read in data
prot, rna, one_hot_cls, one_hot_drugs, ic50_df1 = di_nb.read_input_data()
_all_cls = prot.index
_all_drugs = ic50_df1.columns

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Number of missing prot values 0.386335609896865
num non overlapping prot and target cls: 10
num non overlapping rna prot and target cls: 91


In [6]:
prot.shape, rna.shape, one_hot_cls.shape, one_hot_drugs.shape

((877, 8457), (877, 17417), (877, 877), (345, 345))

## Feature selection (FS) and creating data for each drug

### RNA FS

In [7]:
#read in landmark genes for fs and find landmarks that overlap with rna data
landmark_genes = pd.read_csv(
    f'{codebase_path}/downloaded_data_small/landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

dft = pd.DataFrame(rna.columns.dropna())
dft.index = rna.columns.dropna()
dft = dft[dft.duplicated() == False]

overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), dft)

overlapping_landmarks = overlapping_landmarks['Symbol'].values

#create input data for each drug
x_all, x_drug, y_list = dp_nb.create_all_drugs(
    rna[overlapping_landmarks], one_hot_drugs, ic50_df1, _all_cls)

x_all = x_all.astype(np.float32)
x_drug = x_drug.astype(np.float16)

#fmt index to include drug cell line paris
cls_drugs_index = x_all.index + '::' + x_drug.index
x_all.index = cls_drugs_index
x_drug.index = cls_drugs_index
y_list.index = cls_drugs_index

x_all.shape, x_drug.shape, len(y_list)

((263375, 908), (263375, 345), 263375)

### Prot FS

In [8]:
#use the same landmark genes, that were used for fs for rna data
#for fs with prot data
#find overlapping landmark genes and prot features
dft = pd.DataFrame(prot.columns.dropna())
dft.index = prot.columns.dropna()
dft = dft[dft.duplicated() == False]

overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), dft)

overlapping_landmarks = overlapping_landmarks['Symbol'].values

#create prot data for all drugs
x_all_prot, x_drug, y_list = dp_nb.create_all_drugs(
    prot[overlapping_landmarks], one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_prot.index + '::' + x_drug.index 
x_all_prot.index = cls_drugs_index
y_list.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_prot = x_all_prot.astype(np.float32)

### Create one hot data for all drugs

In [9]:
x_hot, x_drug_hot, y_hot = dp_nb.create_all_drugs(
    one_hot_cls, one_hot_drugs, ic50_df1, _all_cls)

cls_drugs_index_hot = x_hot.index + '::' + x_drug_hot.index 

x_hot.index = cls_drugs_index_hot

# Learning curves 

## Model buliding

In [10]:
_input_shape=None
def build_cnn_kt(hp):
    if _input_shape == None:
        raise Exception('add input shape dim')
    phos_input = layers.Input(shape=(_input_shape, 1))
    x = layers.Conv1D(
        filters=hp.Int('filts', 8, 32, 8), kernel_size=16, 
        activation='relu')(phos_input)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(
        filters=hp.Int('filts',8, 32, 8), kernel_size=8, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(hp.Int('units', 32, 258, 32), activation='relu')(x)
    x = layers.Dense(hp.Int('units', 32, 258, 32), activation='relu')(x)
    drug_input = layers.Input(shape = (xdrug_train.shape[1]))
    concatenated = layers.concatenate([x, drug_input])
    hidd = layers.Dense(hp.Int('units_hid', 32, 258, 32), activation='relu')(concatenated)
    hidd = layers.Dense(hp.Int('units_hid', 32, 258, 32), activation='relu')(hidd)
    output_tensor = layers.Dense(1)(hidd)
    model = tf.keras.Model([phos_input,drug_input], output_tensor)
    opt = tf.keras.optimizers.RMSprop(learning_rate=hp.Choice('lr', [1e-4, 1e-3, 1e-2]))
    model.compile(optimizer=opt, loss=tf.keras.metrics.mean_squared_error, metrics=['mae'])
    return model

In [11]:
#just to get train size
rand_seed = 1
pairs_with_truth_vals =  y_list.index
train_pairs, test_pairs, val_pairs = tts_nb.split(
    rand_seed, _all_cls, _all_drugs, pairs_with_truth_vals)


#set train size search space. 
lg_space = np.logspace(1, 17.6, base=2.0).astype(int)
lg_space = np.append(lg_space, len(test_pairs))
lg_space = np.unique(lg_space)
lg_space

Fraction of cls in sets, relative to all clsbefore mising values are removed
train fraction 0.7993158494868872, test fraction 0.10034207525655645,validaiton fraciton 0.10034207525655645
------
Fraction of cls in sets, relative to all cl drug pairs, aftermising values are removed
train fraction 0.6972253895857089, test fraction0.08817939946788293, validaiton fraciton 0.0850693239469205


array([     2,      3,      4,      5,      6,      8,     10,     13,
           16,     20,     26,     33,     42,     53,     67,     85,
          108,    136,    173,    219,    277,    350,    443,    560,
          708,    896,   1133,   1433,   1813,   2293,   2900,   3668,
         4638,   5866,   7419,   9383,  11867,  15008,  18980,  24004,
        26680,  30358,  38393,  48555,  61407,  77660,  98216, 124212,
       157089, 198668])

# Learning curve runs

In [12]:
t1 = time.time()

In [ ]:
#RNA
#finds a test train split then finds the learning curve
#for that split. Repeats for mutiple (N) test train splits 
N = 30

for run in range(25, N):
    print(f'run {run} of {N}')
    #test train split
    rand_seed = 42 + run
    pairs_with_truth_vals =  y_list.index
    train_pairs, test_pairs, val_pairs = tts_nb.split(
        rand_seed, _all_cls, _all_drugs, pairs_with_truth_vals)

    #rna test train selection
    x_train_rna, x_test_rna = x_all.loc[train_pairs], x_all.loc[test_pairs]
    x_val_rna = x_all.loc[val_pairs]
    y_train, y_test = y_list[train_pairs], y_list[test_pairs]
    y_val = y_list[val_pairs]
    xdrug_train, xdrug_test = x_drug.loc[train_pairs], x_drug.loc[test_pairs]
    xdrug_val = x_drug.loc[val_pairs]

    #prot test train selection
    x_train_prot, x_test_prot = x_all_prot.loc[train_pairs], x_all_prot.loc[test_pairs]
    x_val_prot = x_all_prot.loc[val_pairs]

    #one hot test train seleciton
    x_train_hot, x_test_hot = x_hot.loc[train_pairs], x_hot.loc[test_pairs]
    x_val_hot = x_hot.loc[val_pairs]


    #consistencey checks
    assert (x_train_hot.index == x_train_rna.index).all()
    assert (x_test_hot.index == x_test_rna.index).all()
    assert (x_val_hot.index == x_val_rna.index).all()

    assert (x_train_prot.index == x_train_rna.index).all()
    assert (x_test_prot.index == x_test_rna.index).all()
    assert (x_val_prot.index == x_val_rna.index).all()

    assert (y_train.index == x_train_rna.index).all()
    assert (y_test.index == x_test_rna.index).all()
    assert (xdrug_test.index == x_test_rna.index).all()

    #inconsistencey checks
    assert x_train_rna.shape[1] != x_train_prot.shape[1]
    assert x_test_rna.shape[1] != x_test_prot.shape[1]
    assert x_val_rna.shape[1] != x_val_prot.shape[1]

    assert x_train_rna.shape[1] != x_train_hot.shape[1]
    assert x_test_rna.shape[1] != x_test_hot.shape[1]
    assert x_val_rna.shape[1] != x_val_hot.shape[1]

    assert x_train_prot.shape[1] != x_train_hot.shape[1]
    assert x_test_prot.shape[1] != x_test_hot.shape[1]

    #run the learning curve
    _input_shape = x_train_rna.shape[1]
    mse_r2_rna, bms, bhps = lc_nb.run_lc_ucl(
        build_cnn_kt,
        [x_train_rna, xdrug_train], 
        y_train, 
        [x_val_rna, xdrug_val], 
        y_val, 
        [x_test_rna, xdrug_test],
        y_test, 
        lg_space, 
        num_trails=15,
        epochs=100,
        direc='UCL-del1')
    
    #save data
    mse_r2_rna.to_csv(f'LC-metric-results/RNA/run{run}')
    bhps_df = pd.DataFrame([bhp.values for bhp in bhps])
    bhps_df.to_csv(f'Optimal-hyperparameters/RNA/run{run}df')
    with open(f'Optimal-hyperparameters/RNA/run{run}.pkl', 'wb') as f:
        pickle.dump(bhps, f)
    model_path = f'optimal-models/RNA/run{run}/model_train_size_'
    for train_size, model in zip(lg_space, bms):
        model.save(model_path + str(train_size)) 

Trial 2 Complete [00h 00m 06s]
val_loss: 6.717092990875244

Best val_loss So Far: 6.717092990875244
Total elapsed time: 00h 00m 12s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
filts             |8                 |32                
units             |192               |192               
units_hid         |128               |160               
lr                |0.001             |0.01              

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 22.8601 - mae: 4.4802 - val_loss: 46.8660 - val_mae: 6.3374
Epoch 2/100
1/1 [==============================] - 1s 525ms/step - loss: 41.5906 - mae: 5.9012 - val_loss: 11.1734 - val_mae: 2.8967
Epoch 3/100
1/1 [==============================] - 1s 515ms/step - loss: 13.0587 - mae: 3.2197 - val_loss: 8.2008 - val_mae: 2.1015
Epoch 4/100
1/1 [==============================] - 1s 514ms/step - loss: 7.3791 - mae: 1.7758 - val_loss: 7.2198 - val_mae: 2.1871
Epoch 5/100
1/1 [============

## Check loading in the HPs and create models form that
a fair amount different from what I had before. 

In [34]:
hp_path = 'Optimal-hyperparameters/RNA/'
tt_path = 'test_train_cls/'

In [84]:
test_results = {'r2': [], 'mse': [], 'rho' : []}
_input_shape = x_all.shape[1]
for run in range(0, 10):

    train_pairs = pd.read_csv(f'{tt_path}train_pairs{run}', dtype=str, 
                          delimiter='/n',header=None)
    test_pairs = pd.read_csv(f'{tt_path}test_pairs{run}', dtype=str, 
                          delimiter='/n',header=None)
    val_pairs = pd.read_csv(f'{tt_path}val_pairs{run}', dtype=str, 
                          delimiter='/n',header=None)
    train_pairs = train_pairs[0].values
    test_pairs = test_pairs[0].values
    val_pairs  = val_pairs[0].values

    #rna test train selection
    x_train_rna, x_test_rna = x_all.loc[train_pairs], x_all.loc[test_pairs]
    x_val_rna = x_all.loc[val_pairs]
    y_train, y_test = y_list[train_pairs], y_list[test_pairs]
    y_val = y_list[val_pairs]
    xdrug_train, xdrug_test = x_drug.loc[train_pairs], x_drug.loc[test_pairs]
    xdrug_val = x_drug.loc[val_pairs]

    #create and fit model with final data size.
    f = open(f'{hp_path}run{run}.pkl', 'rb')
    hp_load = pickle.load(f)
    f.close()
    model = build_cnn_kt(hp_load[-1])

    
    callbacks = [EarlyStopping(monitor='val_loss', patience=5, 
                               restore_best_weights=True)]



    model.fit([x_train_rna, xdrug_train], y_train,
              validation_data=([x_val_rna, xdrug_val], y_val),
              epochs=100,
              callbacks=callbacks,
              batch_size=128
        )

    #make predctions using the opt model
    pre = model.predict([x_test_rna, xdrug_test])
    pre = pre.reshape(len(pre))
    test_results['rho'].append(scipy.stats.pearsonr(y_test, pre))
    test_results['r2'].append(sklearn.metrics.r2_score(y_test, pre))
    test_results['mse'].append(sklearn.metrics.mean_squared_error(y_test, pre))


/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Epoch 1/100
1640/1640 [==============================] - 10s 6ms/step - loss: 5.2457 - mae: 1.7343 - val_loss: 2.8232 - val_mae: 1.2814
Epoch 2/100
1640/1640 [==============================] - 9s 6ms/step - loss: 1.8727 - mae: 1.0090 - val_loss: 1.9114 - val_mae: 1.0338
Epoch 3/100
1640/1640 [==============================] - 10s 6ms/step - loss: 1.5321 - mae: 0.9166 - val_loss: 1.7405 - val_mae: 0.9885
Epoch 4/100
1640/1640 [==============================] - 9s 6ms/step - loss: 1.4755 - mae: 0.8969 - val_loss: 1.7218 - val_mae: 0.9762
Epoch 5/100
1640/1640 [==============================] - 9s 6ms/step - loss: 1.4346 - mae: 0.8828 - val_loss: 1.7652 - val_mae: 0.9938
Epoch 6/100
1640/1640 [==============================] - 9s 6ms/step - loss: 1.3886 - mae: 0.8669 - val_loss: 1.6819 - val_mae: 0.9693
Epoch 7/100
1640/1640 [==============================] - 9s 6ms/step - loss: 1.3421 - mae: 0.8509 - val_loss: 1.6518 - val_mae: 0.9559
Epoch 8/100
1640/1640 [==============================

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Epoch 1/100
1640/1640 [==============================] - 11s 6ms/step - loss: 4.0560 - mae: 1.4841 - val_loss: 1.9843 - val_mae: 1.0529
Epoch 2/100
1640/1640 [==============================] - 10s 6ms/step - loss: 1.5375 - mae: 0.9182 - val_loss: 1.7254 - val_mae: 0.9761
Epoch 3/100
1640/1640 [==============================] - 10s 6ms/step - loss: 1.4282 - mae: 0.8825 - val_loss: 2.1023 - val_mae: 1.1083
Epoch 4/100
1640/1640 [==============================] - 10s 6ms/step - loss: 1.3426 - mae: 0.8522 - val_loss: 1.6670 - val_mae: 0.9537
Epoch 5/100
1640/1640 [==============================] - 10s 6ms/step - loss: 1.2810 - mae: 0.8294 - val_loss: 1.6510 - val_mae: 0.9483
Epoch 6/100
1640/1640 [==============================] - 10s 6ms/step - loss: 1.2324 - mae: 0.8120 - val_loss: 1.6413 - val_mae: 0.9521
Epoch 7/100
1640/1640 [==============================] - 10s 6ms/step - loss: 1.1918 - mae: 0.7969 - val_loss: 1.6832 - val_mae: 0.9620
Epoch 8/100
1640/1640 [=========================

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


...

Epoch 1/100
1648/1648 [==============================] - 10s 6ms/step - loss: 4.1648 - mae: 1.5095 - val_loss: 2.4131 - val_mae: 1.1780
Epoch 2/100
1648/1648 [==============================] - 10s 6ms/step - loss: 1.5905 - mae: 0.9359 - val_loss: 1.9557 - val_mae: 1.0424
Epoch 3/100
1648/1648 [==============================] - 10s 6ms/step - loss: 1.4609 - mae: 0.8945 - val_loss: 1.8320 - val_mae: 0.9899
Epoch 4/100
1648/1648 [==============================] - 10s 6ms/step - loss: 1.3995 - mae: 0.8741 - val_loss: 1.8073 - val_mae: 0.9889
Epoch 5/100
1648/1648 [==============================] - 10s 6ms/step - loss: 1.3369 - mae: 0.8529 - val_loss: 1.7191 - val_mae: 0.9586
Epoch 6/100
1648/1648 [==============================] - 10s 6ms/step - loss: 1.2916 - mae: 0.8364 - val_loss: 1.7203 - val_mae: 0.9557
Epoch 7/100
1648/1648 [==============================] - 10s 6ms/step - loss: 1.2571 - mae: 0.8238 - val_loss: 1.7307 - val_mae: 0.9623
Epoch 8/100
1648/1648 [=========================

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Epoch 1/100
1638/1638 [==============================] - 10s 6ms/step - loss: 3.9295 - mae: 1.4571 - val_loss: 1.8795 - val_mae: 1.0156
Epoch 2/100
1638/1638 [==============================] - 10s 6ms/step - loss: 1.5703 - mae: 0.9285 - val_loss: 2.0101 - val_mae: 1.0813
Epoch 3/100
1638/1638 [==============================] - 10s 6ms/step - loss: 1.4617 - mae: 0.8947 - val_loss: 1.7793 - val_mae: 0.9951
Epoch 4/100
1638/1638 [==============================] - 10s 6ms/step - loss: 1.3838 - mae: 0.8671 - val_loss: 1.6238 - val_mae: 0.9446
Epoch 5/100
1638/1638 [==============================] - 10s 6ms/step - loss: 1.3210 - mae: 0.8454 - val_loss: 1.6008 - val_mae: 0.9318
Epoch 6/100
1638/1638 [==============================] - 10s 6ms/step - loss: 1.2784 - mae: 0.8297 - val_loss: 1.6056 - val_mae: 0.9381
Epoch 7/100
1638/1638 [==============================] - 10s 6ms/step - loss: 1.2464 - mae: 0.8177 - val_loss: 1.6326 - val_mae: 0.9419
Epoch 8/100
1638/1638 [=========================

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Epoch 1/100
1651/1651 [==============================] - 11s 6ms/step - loss: 2.3484 - mae: 1.0958 - val_loss: 1.7588 - val_mae: 0.9843
Epoch 2/100
1651/1651 [==============================] - 10s 6ms/step - loss: 1.3214 - mae: 0.8455 - val_loss: 1.6869 - val_mae: 0.9606
Epoch 3/100
1651/1651 [==============================] - 10s 6ms/step - loss: 1.2540 - mae: 0.8201 - val_loss: 1.7494 - val_mae: 0.9822
Epoch 4/100
1651/1651 [==============================] - 10s 6ms/step - loss: 1.2046 - mae: 0.8022 - val_loss: 1.7646 - val_mae: 0.9920
Epoch 5/100
1651/1651 [==============================] - 10s 6ms/step - loss: 1.1569 - mae: 0.7853 - val_loss: 1.7278 - val_mae: 0.9672
Epoch 6/100
1651/1651 [==============================] - 10s 6ms/step - loss: 1.1156 - mae: 0.7711 - val_loss: 1.7007 - val_mae: 0.9593
Epoch 7/100
1651/1651 [==============================] - 10s 6ms/step - loss: 1.0749 - mae: 0.7562 - val_loss: 1.6833 - val_mae: 0.9540
Epoch 8/100
1651/1651 [=========================

In [86]:
pd.DataFrame(test_results)

,r2,mse,rho
0,0.779061,1.522273,"(0.8844822228301752, 0.0)"
1,0.774705,1.547763,"(0.8832897093048311, 0.0)"
2,0.781551,1.477086,"(0.8842145179154487, 0.0)"
3,0.752757,1.660370,"(0.8690738868844304, 0.0)"
4,0.777163,1.519671,"(0.8818154292117533, 0.0)"
5,0.737942,1.742636,"(0.8604518080052003, 0.0)"
6,0.764542,1.576318,"(0.8744653539259567, 0.0)"
7,0.758026,1.645188,"(0.8713530159070254, 0.0)"
8,0.736138,1.785626,"(0.858839100628207, 0.0)"
9,0.758610,1.669756,"(0.8713034845954499, 0.0)"


In [94]:
pd.DataFrame(import_results)

,r2,mse,rho
0,0.782423,1.499113,"(0.8851309570895478, 0.0)"
1,0.768829,1.588129,"(0.877253198022631, 0.0)"
2,0.787669,1.435717,"(0.8876906255774633, 0.0)"
3,0.755087,1.644722,"(0.8702065076396063, 0.0)"
4,0.786594,1.455355,"(0.8872465961928492, 0.0)"
5,0.736280,1.753687,"(0.8612760326017215, 0.0)"
6,0.770250,1.538103,"(0.8779800249678034, 0.0)"
7,0.756510,1.655495,"(0.870798518432726, 0.0)"
8,0.739969,1.759703,"(0.861194891052643, 0.0)"
9,0.762738,1.641199,"(0.8742154490725469, 0.0)"


In [93]:
import_results = {'r2': [], 'mse': [], 'rho' : []}

for run in range(0, 10):
    imp = pd.read_csv(f'LC-metric-results/RNA/run{run}').iloc[-1]
    import_results['r2'].append(imp['r2'])
    import_results['mse'].append(imp['mse'])
    import_results['rho'].append(imp['rho'])

In [73]:
from tensorflow.keras.callbacks import EarlyStopping


In [81]:
pd.read_csv(f'LC-metric-results/RNA/run{run}')

,Unnamed: 0,r2,mse,rho,train size
0,0,-0.007072,6.938752,"(0.01634634781390522, 0.00785402067895004)",2
1,1,-0.006600,6.935496,"(0.05700137924380597, 1.749363358036882e-20)",3
2,2,-0.006564,6.935248,"(0.13720963580815163, 2.62944867484629e-111)",4
3,3,-0.003861,6.916628,"(0.06776613293108544, 2.6714148099086835e-28)",5
4,4,-0.001500,6.900357,"(0.059410176552587735, 4.070782066799802e-22)",6
5,5,-0.003166,6.911840,"(0.06232432691891631, 3.5051639565363745e-24)",8
6,6,0.004450,6.859362,"(0.11308099547545845, 5.464882570654515e-76)",10
7,7,0.001156,6.882057,"(0.10197538093186086, 4.5384869524186375e-62)",13
8,8,0.000861,6.884090,"(0.11681767966359655, 5.305624831804361e-81)",16
9,9,0.001017,6.883018,"(0.09825395222424599, 9.882753041289182e-58)",20
